In [3]:
from difflib import SequenceMatcher
import pandas as pd

In [2]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_closest_matches(df, actual_col, pred_col, thresh):
    actual = df[df[actual_col].notna()][actual_col].values
    preds = df[df[pred_col].notna()][pred_col].values
    
    res = {}
    for x in set(actual):
        tmp = []
        for y in set(preds):
            score = similar(x, y)
            if score >= thresh:
                tmp.append((y, score))
        tmp.sort(key=lambda x: x[1], reverse=True)
        res[x] = tmp
    return res

def replace_closest_matches(df, actual_col, pred_col, col_map, n):
    actual = df[df[actual_col].notna()][actual_col].values
    preds = df[df[pred_col].notna()][pred_col].values
    mp = {}
    
    for a in actual:
        m = min(n, len(col_map.get(a, [])))
        for i in range(m):
            mp[col_map[a][i][0]] = a
            
    df[pred_col] = df[pred_col].apply(lambda x: mp[x] if x in mp else x)
    return mp, df

In [5]:
t1 = pd.read_csv('task1_valid.csv')
t1 = t1.where(pd.notnull(t1), None)

In [6]:
col_map = get_closest_matches(
    t1, 'location', 'pred_location', thresh=0.7,
)
mp, new_df = replace_closest_matches(
    t1, 'location', 'pred_location', col_map, n=3,
)

In [7]:
new_df['location'].unique().shape, new_df['pred_location'].unique().shape

((1336,), (2785,))